# Azure AI Search NVIDIA NIM మరియు LlamaIndex ఇంటిగ్రేషన్

ఈ నోట్‌బుక్‌లో, NVIDIA యొక్క AI మోడల్స్ మరియు LlamaIndex ఉపయోగించి శక్తివంతమైన Retrieval-Augmented Generation (RAG) పైప్‌లైన్‌ను ఎలా ఉపయోగించాలో చూపిస్తాం. NVIDIA యొక్క LLMs మరియు embeddings ని ఉపయోగించి, వాటిని Azure AI Search ను వెక్టర్ స్టోర్ గా సమీకరించి, శోధన నాణ్యత మరియు సమర్థతను మెరుగుపరచడానికి RAG నిర్వర్తిస్తాం.

## లాభాలు
- **Scalability**: పరిమాణం పెరగదగ్గ మరియు సమర్థవంతమైన రిట్రీవల్ కోసం NVIDIA యొక్క పెద్ద భాషా మోడల్స్ మరియు Azure AI Search ను వినియోగించండి.
- **Cost Efficiency**: సమర్థవంతమైన వెక్టర్ నిల్వ మరియు హైబ్రిడ్ శోధన సాంకేతికతలతో శోధన మరియు రిట్రీవల్ ఖర్చులను ఆప్టిమైజ్ చేయండి.
- **High Performance**: వేగవంతమైన మరియు ఖచ్చితమైన ప్రతిస్పందనలు కోసం శక్తివంతమైన LLMs ని వెక్టరైజ్డ్ శోధనతో కలపండి.
- **Quality**: సంబంధిత రికవర్డ్ డాక్యుమెంట్లతో LLM ప్రతిస్పందనలను గ్రౌండింగ్ చేయడం ద్వారా ఉన్నత శోధన నాణ్యతను నిర్వహించండి.

## అవసరాలు
- 🐍 Python 3.9 or higher
- 🔗 [Azure AI Search Service](https://learn.microsoft.com/azure/search/)
- 🔗 NVIDIA API Key for access to NVIDIA's LLMs and Embeddings via the NVIDIA NIM microservices

## కవర్ చేయబడిన ఫీచర్లు
- ✅ NVIDIA LLM Integration (మనం [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe) ను ఉపయోగిస్తాం)
- ✅ NVIDIA Embeddings (మనం [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5) ను ఉపయోగిస్తాం)
- ✅ Azure AI Search యొక్క అధునాతన రిట్రీవల్ మోడ్స్
- ✅ LlamaIndex తో డాక్యూమెంట్ సూచికలు రూపొందించడం
- ✅ NVIDIA LLMs తో Azure AI Search మరియు LlamaIndex ఉపయోగించి RAG

మనం ప్రారంభిద్దాం!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## ఇన్‌స్టాలేషన్ మరియు అవసరాలు
Python వెర్షన్ >3.10 ఉపయోగించి ఒక Python పర్యావరణాన్ని సృష్టించండి.

## ప్రారంభించండి!


ప్రారంభించడానికి, NVIDIA AI Foundation మోడల్స్ ఉపయోగించడానికి మీకు `NVIDIA_API_KEY` అవసరం:
1) [NVIDIA](https://build.nvidia.com/explore/discover)తో ఒక ఉచిత ఖాతాను సృష్టించండి.
2) మీరు ఎంచుకున్న మోడల్‌పై క్లిక్ చేయండి.
3) Input విభాగంలో, Python ట్యాబ్‌ను ఎంచుకుని, **API కీ పొందండి**పై క్లిక్ చేసి, తరువాత **కీ ఉత్పత్తి చేయండి**పై క్లిక్ చేయండి.
4) సృష్టించిన కీని కాపీ చేసి NVIDIA_API_KEY గా సేవ్ చేయండి. ఆ తర్వాత, మీరు ఎండ్పాయింట్‌లకు ప్రాప్యత కలిగి ఉంటారు.


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## LLM మరియు ఎంబెడింగ్ ఉపయోగించి RAG ఉదాహరణ
### 1) LLMని ప్రారంభించండి
`llama-index-llms-nvidia`, NVIDIA యొక్క LLM కనెక్టర్ గా కూడా పిలవబడే ఇది, NVIDIA API క్యాటలాగ్‌లో అందుబాటులో ఉన్న అనుకూల మోడల్స్‌కు కనెక్ట్ అవ్వడానికి మరియు వాటి నుండి తయారు చేయడానికి అనుమతిస్తుంది. చాట్ కంప్లీషన్ మోడల్స్ జాబితా కోసం ఇక్కడ చూడండి: https://build.nvidia.com/search?term=Text-to-Text

ఇక్కడ మనం **mixtral-8x7b-instruct-v0.1** ను ఉపయోగించబోతున్నాం


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) ఎంబెడ్డింగ్‌ను ప్రారంభించండి
`llama-index-embeddings-nvidia`, దీనిని NVIDIA యొక్క Embeddings కనెక్టర్‌గా కూడా పిలుస్తారు, ఇది NVIDIA API క్యాటలాగ్‌లో అందుబాటులో ఉన్న అనుకూల మోడళ్లకు కనెక్ట్ అయ్యి అవి నుండి జనరేట్ చేయడానికి మీకు అనుమతిస్తుంది. మేము ఎంబెడ్డింగ్ మోడల్‌గా `nvidia/nv-embedqa-e5-v5`ని ఎంచుకున్నాము. టెక్స్ట్ ఎంబెడ్డింగ్ మోడళ్ల జాబితా కోసం ఇక్కడ చూడండి: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

### 3) Azure AI Search వెక్టర్ స్టోర్‌ను సృష్టించండి


In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

### 4) లోడ్ చేయండి, భాగాలుగా విభజించండి, మరియు డాక్యుమెంట్లను అప్లోడ్ చేయండి


In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) మీ డేటాపై ప్రశ్నలు అడగడానికి ఒక క్వెరీ ఇంజిన్ సృష్టించండి

ఇది Azure AI Searchలో శుద్ధ వెక్టర్ శోధనను ఉపయోగించి, సమాధానాన్ని మా LLM (Phi-3.5-MOE)తో గ్రౌండింగ్ చేసే ఒక క్వెరీ.


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

ఇది Azure AI Searchలో హైబ్రిడ్ సెర్చ్ ఉపయోగించే ఒక క్వెరీ.


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### వెెక్టర్ సెర్చ్ విశ్లేషణ
LLM ప్రతిస్పందన రష్యా స్టాక్ మార్కెట్ గురించి మూల పాఠ్యంలోని ప్రస్తావించిన కీలక ఆర్థిక పరిణామాలను ఖచ్చితంగా పట్టుకుంది. ఖచ్చితంగా, ఇది రష్యా స్టాక్ మార్కెట్ దాదాపుగా 40% విలువ కోల్పోయి గణనీయంగా పడిపోయిందని, అలాగే కొనసాగుతున్న పరిస్థితుల కారణంగా వాణిజ్యం నిలిపివేయబడిందని పేర్కొంటుంది. ఈ ప్రతిస్పందన మూలంలో ఉన్న సమాచారానికి బాగా సరిపోతుంది, ఇది LLM రష్యా చర్యలు మరియు విధించిన ఆంక్షల ఫలితంగా స్టాక్ మార్కెట్ పై వచ్చిన ప్రభావానికి సంబంధించి సంబంధిత వివరాలను సరిగ్గా గుర్తించి సంక్షిప్తంగా అందించినట్టే సూచిస్తుంది.

#### మూల నోడ్స్ వ్యాఖ్యానం
మూల నోడ్స్ అంతర్జాతీయ ఆంక్షల కారణంగా రష్యాకు ఎదురైన ఆర్థిక పరిణామాల గురించి వివరమైన వివరణను అందించాయి. ఈ టెక్స్ట్ రష్యా స్టాక్ మార్కెట్ తన విలువలో 40% నష్టపొందినదని, మరియు వాణిజ్యం నిలిపివేయబడినదని హైలైట్ చేస్తుంది. అదనంగా, ఇది రూబల్ విలువ తగ్గిపోవడం వంటి ఇతర ఆర్థిక ప్రభావాలను మరియు రష్యా ఆర్థిక వ్యవస్థ యొక్క విస్తృత ఏకాంతీకరణను కూడా ప్రస్తావిస్తుంది. ఈ నోడ్స్ నుండి కీలక అంశాలను LLM ప్రతిస్పందన సమర్థవంతంగా సంక్షిప్తం చేసింది, ప్రశ్నలో అభ్యర్థించినట్లుగా స్టాక్ మార్కెట్ ప్రభావంపై దృష్టి పెట్టింది.


ఇప్పుడు, Hybrid Search బలమైన ఆధారాలతో కూడిన జవాబు ఇవ్వని ఒక క్వెరీని చూద్దాం:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### హైబ్రిడ్ సెర్చ్: LLM ప్రతిస్పందన విశ్లేషణ
హైబ్రిడ్ సెర్చ్ ఉదాహరణలోని LLM ప్రతిస్పందన ఇవ్వబడిన సందర్భం రష్యా యొక్క ఉక్రెయిన్‌పై ఆక్రమణ యొక్క ఖచ్చిత తేదీని నిర్దేశించదని సూచిస్తుంది. ఈ ప్రతిస్పందన LLM మూల పత్రాల్లో అందుబాటులో ఉన్న సమాచారాన్ని వినియోగిస్తున్నట్లు సూచిస్తేనూ, పాఠ్యంలో ఖచ్చిత వివరాల లేమును కూడా అంగీకరిస్తుంది.

ఈ ప్రతిస్పందన సందర్భం రష్యా దాడికి సంబంధించిన సంఘటనలను పేర్కొనడం సరైనదని అయితే నిర్దిష్ట ఆక్రమణ తేదీని గుర్తించలేదని సరిగ్గా గుర్తిస్తుంది. ఇది అందించిన సమాచారాన్ని అర్ధం చేసుకునే LLM యొక్క సామర్థ్యాన్ని మరియు విషయాల్లో ఉన్న ఖాళీలను గుర్తించే సామర్ధ్యాన్ని చూపిస్తుంది. సమాచారం అసంపూర్ణంగా ఉండగా ఖచ్చిత తేదీ కోసం బాహ్య వనరులు లేదా చారిత్రాత్మక రికార్డులను చూడమంటూ LLM ప్రభావవంతంగా యూజర్‌ను ప్రేరేపిస్తుంది, ఇది జాగ్రత్తగా వ్యవహరించే స్వభావాన్ని ప్రదర్శిస్తుంది.

### మూల నోడ్‌ల విశ్లేషణ
హైబ్రిడ్ సెర్చ్ ఉదాహరణలోని మూల నోడ్‌లు ఉక్రెయిన్‌లో రష్యా చర్యలకు అమెరికా ప్రతిస్పందన గురించి జరిగిన ఒక ప్రసంగం నుండి తీసుకున్న ఉద్భాగాలను కలిగి ఉంటాయి. ఈ నోడ్‌లు ఆక్రమణకు ప్రతిస్పందనగా అమెరికా మరియు దాని మైత్రి రాష్ట్రాలు తీసుకున్న చర్యలు మరియు విస్తృత జియోపాలిటికల్ ప్రభావాన్ని ప్రాముఖ్యంగా తెలియజేస్తాయి, అయితే అవి నిర్దిష్ట ఆక్రమణ తేదీని పేర్కొనవు. ఇది సందర్భంలో ఖచ్చిత తేదీ సమాచారాన్ని తగ్గిపోవడం అని సరిగా గుర్తించిన LLM ప్రతిస్పందనతో అనుగుణంగా ఉంది.


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### రీర్యాంకింగ్‌తో హైబ్రిడ్: LLM ప్రతిస్పందన విశ్లేషణ
హైబ్రిడ్ రీర్యాంకింగ్ ఉదాహరణలో, LLM ప్రతిస్పందన భాషణం ఇచ్చే సమయానికి ఆ సంఘటన ఆరు రోజుల ముందు జరిగింది అని గమనించడం ద్వారా అదనపు పరిసర సమాచారం అందిస్తుంది. ఇది LLM భాషణ సమయాన్ని ఆధారంగా ఆక్రమణ తేదీని ఊహించగలగడం సూచిస్తుంది, అయితే ఖచ్చితత్వానికి భాషణ యొక్క ఖచ్చితమైన తేదీ ఇంకా అవసరం.

ఈ ప్రతిస్పందన పర్యావరణ సంకేతాలను ఉపయోగించి మరింత సమాచారమయమైన సమాధానాన్ని ఇవ్వడానికి మెరుగైన సామర్థ్యాన్ని ప్రదర్శిస్తుంది. ఇది రీర్యాంకింగ్ యొక్క ప్రయోజనాన్ని హైలైట్ చేస్తుంది, అక్కడ LLM మరింత సంబంధిత సమాచారాన్ని యాక్సెస్ చేసి ప్రాధాన్యత ఇవ్వగలదు తద్వారా కావలసిన వివరానికి (ఉదాహరణకి, ఆక్రమణ తేదీ) సమీపమైన అంచనాను ఇవ్వగలదు.

### మూల నోడ్స్ విశ్లేషణ
ఈ ఉదాహరణలోని మూల నోడ్స్ రష్యా ఆక్రమణ సమయానికి సంబంధించిన సూచనలను కలిగి ఉన్నాయి, ముఖ్యంగా అది భాషణానికి ఆరు రోజుల ముందు జరిగింది అని ప్రస్తావించడం. ఖచ్చితమైన తేదీ ఇంకా స్పష్టంగా ఇవ్వబడొని ఉన్నప్పటికీ, ఈ నోడ్స్ సమయ సంబంధ పరిసర సమాచారాన్ని అందిస్తాయి, దాని ద్వారా LLM మరింత సూక్ష్మత అంతచేతనతో సమాధానం ఇవ్వగలదు. ఈ వివరాన్ని చేర్చడం రీర్యాంకింగ్ ఎలా LLM యొక్క అందుబాటులో ఉన్న పరిసర సమాచారాన్ని ఎగ్జ్రాక్ట్ చేసి అంచనా వేయడంలో మెరుగుపరచగలదో, ఫలితంగా మరింత ఖచ్చితమైన మరియు సమాచారమయమైన సమాధానం రావడానికిని చూపిస్తుంది.


**Note:**
ఈ నోట్బుక్‌లో, మేము NVIDIA API క్యాటలాగ్‌లోని NVIDIA NIM మైక్రోసర్వీసులను ఉపయోగించాము.
పైన పేర్కొన్న APIs, `NVIDIA (llms)`, `NVIDIAEmbedding`, మరియు [Azure AI శోధన సేమాంటిక్ హైబ్రిడ్ రిట్రీవల్ (బిల్ట్-ఇన్ రీరాంకింగ్)](https://learn.microsoft.com/azure/search/semantic-search-overview). గమనిక: పైన పేర్కొన్న APIs స్వయంగా హోస్ట్ చేయబడిన మైక్రోసర్వీసులను కూడా మద్దతు ఇవ్వగలవు। 

**Example:**
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
నిరాకరణ:
ఈ పత్రం కృత్రిమ మేధా (AI) అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ద్వారా అనువదించబడింది. మేము ఖచ్చితత్వాన్ని లక్ష్యంగా పెట్టుకున్నప్పటికీ, స్వయంచాలక అనువాదాల్లో తప్పులు లేదా లోపాలు ఉండే అవకాశం ఉందని దయచేసి గమనించండి. మూల పత్రాన్ని దాని స్థానిక/మాతృ భాషలో ఉన్న రూపాన్ని అధికారిక మూలంగా తీసుకోవాలి. కీలకమైన సమాచారానికి వృత్తిపరమైన మానవ అనువాదాన్ని సిఫార్సు చేయబడుతుంది. ఈ అనువాదాన్ని ఉపయోగించడం వల్ల ఏర్పడే ఏవైనా అవగాహనా లోపాలు లేదా తప్పుగా అర్థం చేసుకోవడాలకు మేము బాధ్యత వహించము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
